In [1]:
%%capture 
!pip install h2o==3.40.0.1

In [2]:
import numpy as np
import pandas as pd
import h2o

In [3]:
def generate_artificial_peptide(list_of_probabilities: np.ndarray, amino_acids: np.ndarray, max_length=22) -> str:
    """
    Generate an artificial peptide based on a list of probabilities and amino acids.
    
    Parameters:
    ----------
    list_of_probabilities : numpy.ndarray
        2-D array of probability of amino acids in the peptide
    amino_acids : numpy.ndarray
        1-D array of amino acids.
        
    Returns:
    -------
    str
        Generated artificial peptide
        
    Notes:
    ------
    The length of the probability array should be same as the length of the peptide.
    """
    out_str = ''
    for i in range(len(list_of_probabilities)):
        # make synthetic signal peptide
        artificial_amino_acid = list(np.random.choice(amino_acids, 1, p=list_of_probabilities[i]))

        if artificial_amino_acid == ['-']: 
            break

        out_str += artificial_amino_acid[0]
    return out_str

def add_dunder_tail(peptide:str , max_lenght:int=22 ): 
    '''Adds a tail if a peptide is shorter than the specified max_len.
    '''
    if len(peptide) < max_lenght: 
        difference = max_lenght - len(peptide)
        sequence = peptide + ('-'*difference)
    else: 
        sequence = peptide
        
    return sequence      

def generate_artificial_peptides(list_of_probabilities: np.ndarray, amino_acids: np.ndarray, n_peptides: int, max_len=50) -> pd.DataFrame:
    """
    Generate a dataframe of artificial peptides based on a list of probabilities and amino acids.
    
    Parameters:
    ----------
    list_of_probabilities : numpy.ndarray
        2-D array of probability of amino acids in the peptide
    amino_acids : numpy.ndarray
        1-D array of amino acids.
    n_peptides : int
        Number of peptides to generate
        
    Returns:
    -------
    pd.DataFrame
        Dataframe of generated artificial peptides with 'sequence' as column
        
    Notes:
    ------
    The length of the probability array should be same as the length of the peptide.
    """
    artificial_peptides = []
    lengths = [] 
    for i in range(n_peptides): 
        peptide = generate_artificial_peptide(list_of_probabilities,amino_acids, max_length=max_len)
        if len(peptide) <= max_len:
            peptide_w_tail = add_dunder_tail(peptide, max_lenght = max_len)
        else: 
            continue
        
        # save
        lengths.append(len(peptide))                                     
        artificial_peptides.append(peptide_w_tail)

    df = pd.DataFrame(artificial_peptides, columns =['sequence'])
    df['length'] = lengths
    return df


def split_peptides_sequences(df_signalPP: pd.DataFrame) -> pd.DataFrame:
    '''
    Splits the amino acid sequences into individual amino acids for each position.

    Parameters:
    -----------
    df_signalPP: pandas.DataFrame
        A DataFrame containing the amino acid sequences.

    Returns:
    --------
    pandas.DataFrame
        A DataFrame containing the split amino acid sequences.
    '''
    # Initialize an empty list to store the split sequences.
    peptides_split = []

    # Split each amino acid sequence into individual amino acids.
    for k, v in df_signalPP.iterrows():
        sequence = []
        for seq in v['sequence']:
            sequence.append(seq)
        peptides_split.append(sequence)

    # Convert the list of split sequences into a DataFrame and fill NaN values with '-'.
    new_peptides = pd.DataFrame(peptides_split)
    new_peptides = new_peptides.fillna('-')

    return new_peptides


def signal_peptide_predictor(list_of_probabilities: list, amino_acids: str, n_peptides: int, number_of_iterations: int, best_model: h2o.estimators, 
                             training_column_name:str = 'MM_N_peptide_abundance', max_len_of_signal_peptides = 30) -> pd.DataFrame:
    '''
    Predicts the best signal peptides from a given number of iterations.

    Parameters:
    -----------
    list_of_probabilities: list
        A list of probabilities.
    amino_acids: str
        A string containing the amino acids used to generate the peptides.
    n_peptides: int
        The number of peptides to generate.
    number_of_iterations: int
        The number of iterations to run the predictor.
    best_model: h2o.estimators
        The trained model to use for prediction.
    training_column_name : str
        The name of the column that the model has been trained on.
    max_len_of_signal_peptides : int
        The maximum lenght of the signal peptides you want to be generated 

    Returns:
    --------
    pandas.DataFrame
        A DataFrame containing the predicted signal peptides.
    '''
    # Initialize an empty DataFrame to store the predicted peptides.
    data = pd.DataFrame()
    
    # Generate and predict peptides for each iteration.
    for i in range(0, number_of_iterations):
        # Generate new peptides.
        new_TO_NATURE_peptides = generate_artificial_peptides(list_of_probabilities, amino_acids, n_peptides=n_peptides, max_len=max_len_of_signal_peptides)
        
        # Split the peptides into sequences.
        new_TO_NATURE_peptides = split_peptides_sequences(new_TO_NATURE_peptides)

        # Convert the DataFrame to an H2OFrame and make the columns categorical.
        df_test = h2o.H2OFrame(pd.concat([new_TO_NATURE_peptides], axis='columns'))
        for column in df_test.columns:
            if column != training_column_name:
                df_test[column] = df_test[column].asfactor()

        # Make predictions on the test data.
        predicted = best_model.predict(df_test).as_data_frame()
        new_TO_NATURE_peptides['predictions'] = predicted['predict'].to_list()

        # Concatenate the new predictions with the existing DataFrame.
        if len(data) == 0:
            data = new_TO_NATURE_peptides.copy()
        else:
            data = pd.concat([data, new_TO_NATURE_peptides], axis=0)
            data = data.sort_values('predictions', ascending=False)
            data = data[0:100]
    
    return data

# Gerating artificial Signal Peptides (SPs)

## 0 Introduction 
To avoid the combinatorial explosion that arises from the large number of possible amino acid sequences, an algorithm must be developed to narrow down the search space and identify the sequences that are most likely to function as signal peptides. This can be accomplished through a variety of computational methods, such as bioinformatics, machine learning, and statistical analysis.

One common approach is to use bioinformatics methods to analyze large sets of data on known signal peptides and identify patterns or features that are associated with signal peptide function. These features can then be used to predict the function of novel sequences.

Machine learning algorithms can also be used to predict signal peptides. These algorithms can be trained on large sets of data on known signal peptides, and can then be used to predict the function of novel sequences. Common machine learning algorithms used for this purpose include decision trees, random forests, and neural networks.

Another approach is to use statistical analysis to identify the regions of the peptide sequences that are most likely to function as signal peptides. This can be done by analyzing the frequency and distribution of different amino acids in known signal peptides and identifying those that are over-represented or under-represented in these sequences.

In summary, by developing an algorithm, we can narrow down the search space and identify the sequences that are most likely to function as signal peptides, thus avoiding combinatorial explosion. The algorithm we are showcasing here is based on a combination of bioinformatics, machine learning and statistical analysis.

### Agenda
- Use AutoML predictions and synthetic signal peptide generation algorotihm to get a novel list of potential signal peptides

## 1 Amino acid probability matrix

Lets import our df_pwn that was made in a previous notebook:

In [4]:
df_pwn = pd.read_csv('../data/02_all_signal_peptides/df_pwn_for_signal_peptides_found_in_supernatant.csv', index_col = False)
df_pwn

,A,C,D,E,F,G,H,I,K,L,...,N,P,Q,R,S,T,V,W,Y,-
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.070312,0.000000,0.003906,0.003906,0.035156,0.000000,0.062500,0.035156,0.234375,0.078125,...,0.011719,0.031250,0.070312,0.214844,0.031250,0.007812,0.046875,0.015625,0.015625,0.000000
2,0.042969,0.000000,0.003906,0.007812,0.179688,0.058594,0.000000,0.050781,0.015625,0.210938,...,0.023438,0.027344,0.023438,0.066406,0.140625,0.050781,0.066406,0.007812,0.015625,0.000000
3,0.070312,0.000000,0.003906,0.000000,0.097656,0.035156,0.015625,0.046875,0.062500,0.167969,...,0.031250,0.042969,0.058594,0.042969,0.179688,0.085938,0.015625,0.011719,0.019531,0.000000
4,0.109375,0.007812,0.007812,0.007812,0.035156,0.027344,0.015625,0.066406,0.066406,0.136719,...,0.046875,0.035156,0.046875,0.015625,0.167969,0.078125,0.046875,0.019531,0.046875,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,0.003906,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.003906,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.992188
64,0.003906,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.996094
65,0.003906,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.996094
66,0.003906,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.996094


In [5]:
amino_acids = list(df_pwn.columns.values)
amino_acids

['A',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'K',
 'L',
 'M',
 'N',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'V',
 'W',
 'Y',
 '-']

In [6]:
list_of_probabilities = []
for i in range(len(df_pwn)): 
    list_of_probabilities.append(df_pwn.loc[i, :].values.tolist())
list_of_probabilities[1]

[0.0703125,
 0.0,
 0.00390625,
 0.00390625,
 0.03515625,
 0.0,
 0.0625,
 0.03515625,
 0.234375,
 0.078125,
 0.03125,
 0.01171875,
 0.03125,
 0.0703125,
 0.21484375,
 0.03125,
 0.0078125,
 0.046875,
 0.015625,
 0.015625,
 0.0]

## 2 Load best model from AutoML

In [7]:
h2o.init(ip="localhost", min_mem_size_GB=8)

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_361"; Java(TM) SE Runtime Environment (build 1.8.0_361-b09); Java HotSpot(TM) 64-Bit Server VM (build 25.361-b09, mixed mode)
  Starting server from /Users/lucaslevassor/opt/anaconda3/envs/constrain/lib/python3.8/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/2f/lw3sfzbs7l7f_q1knzbtbwrr0000gp/T/tmp43gzyfit
  JVM stdout: /var/folders/2f/lw3sfzbs7l7f_q1knzbtbwrr0000gp/T/tmp43gzyfit/h2o_lucaslevassor_started_from_python.out
  JVM stderr: /var/folders/2f/lw3sfzbs7l7f_q1knzbtbwrr0000gp/T/tmp43gzyfit/h2o_lucaslevassor_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,20 secs
H2O_cluster_timezone:,Europe/Copenhagen
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.1
H2O_cluster_version_age:,1 month and 6 days
H2O_cluster_name:,H2O_from_python_lucaslevassor_rk6jkh
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.576 Gb
H2O_cluster_total_cores:,0
H2O_cluster_allowed_cores:,0
H2O_cluster_status:,"locked, healthy"


In [8]:
#h2o.cluster().shutdown()



In [9]:
best_model = h2o.load_model("../data/04_ML_models/0803_m30_filtered_DeepLearning_grid_1_AutoML_2_20230308_164525_model_14.model")
best_model

Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_grid_1_AutoML_2_20230308_164525_model_14


Status of Neuron Layers: predicting abundance, regression, gaussian distribution, Quadratic loss, 59,701 weights/biases, 708.6 KB, 279,837 training samples, mini-batch size 1
    layer    units    type              dropout    l1    l2    mean_rate              rate_rms               momentum    mean_weight            weight_rms          mean_bias             bias_rms
--  -------  -------  ----------------  ---------  ----  ----  ---------------------  ---------------------  ----------  ---------------------  ------------------  --------------------  -----------------------
    1        595      Input             15.0
    2        100      RectifierDropout  40.0       0.0   0.0   0.08383776184387944    0.22281700372695923    0.0         -0.024995468157608204  0.0911882221698761  0.2398722263720472    0.14759206771850586
    3        1        Linear                       0.0   0.0   0.0015805453458415286  0.0027047647163271904  0.0         0.06839000282227062    0.147660493850708   -0.09775900897466194  1.0971281125650402e-154

ModelMetricsRegression: deeplearning
** Reported on train data. **

MSE: 0.0006588389682246397
RMSE: 0.02566785866067989
MAE: 0.0024433722974568006
RMSLE: 0.018117105551137377
Mean Residual Deviance: 0.0006588389682246397

ModelMetricsRegression: deeplearning
** Reported on cross-validation data. **

MSE: 0.0014665389530156953
RMSE: 0.038295416866978946
MAE: 0.00556754191609704
RMSLE: 0.02895905723013147
Mean Residual Deviance: 0.0014665389530156953

Cross-Validation Metrics Summary: 
                        mean        sd          cv_1_valid    cv_2_valid    cv_3_valid    cv_4_valid    cv_5_valid    cv_6_valid    cv_7_valid    cv_8_valid    cv_9_valid    cv_10_valid
----------------------  ----------  ----------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  -------------
mae                     0.0056371   0.00324635  0.0118159     0.010583      0.00585244    0.00695106    0.004379      0.00414332    0.003345      0.00427296    0.00254726    0.00248104
mean_residual_deviance  0.00145843  0.00296455  0.00974658    0.00172341    0.00117409    0.000772287   0.000491289   0.000395702   0.000149314   3.01591e-05   6.2273e-05    3.9206e-05
mse                     0.00145843  0.00296455  0.00974658    0.00172341    0.00117409    0.000772287   0.000491289   0.000395702   0.000149314   3.01591e-05   6.2273e-05    3.9206e-05
r2                      -0.222726   0.613278    0.00516749    -0.00454644   -0.01919      -0.0130522    -0.0227253    -0.018934     -0.013826     -1.96566      -0.101024     -0.0734686
residual_deviance       0.00145843  0.00296455  0.00974658    0.00172341    0.00117409    0.000772287   0.000491289   0.000395702   0.000149314   3.01591e-05   6.2273e-05    3.9206e-05
rmse                    0.0276215   0.0277985   0.0987247     0.041514      0.0342649     0.0277901     0.022165      0.0198923     0.0122194     0.00549173    0.00789132    0.00626147
rmsle                   0.0226405   0.0189134   0.0682184     0.0348133     0.0296352     0.0246152     0.0201131     0.0181916     0.0115827     0.0054735     0.00766329    0.0060985

Scoring History: 
    timestamp            duration           training_speed    epochs    iterations    samples    training_rmse    training_deviance    training_mae    training_r2
--  -------------------  -----------------  ----------------  --------  ------------  ---------  ---------------  -------------------  --------------  -------------
    2023-03-08 17:43:13  0.000 sec                            0         0             0          nan              nan                  nan             nan
    2023-03-08 17:43:13  57 min 39.857 sec  8597 obs/sec      3         1             3009       0.0377321        0.00142371           0.00899153      0.0318429
 

## 3 Signal peptide predictor algorithm

The signal peptide predictor algorithm works by: 

    1. Generating artificial signal peptides - based on the PWM.
    2. It splits the artificial signal peptides into a dataframe adding '_' to the positions that are empty. 
    3. Initiates the H2O dataframe and makes all values except the feature column categorical
    4. Predicts the columns - based on a previously made ML model
    5. Makes a new dataframe with the best predictions, sorts it and slices out top100 
    6. The loop will continue until the specified number is reached. 
 

In [10]:
%%capture

best_predicted_SPs = signal_peptide_predictor(list_of_probabilities, 
                                                        amino_acids, n_peptides =  1000,  
                                                        number_of_iterations = 1000, 
                                                       best_model = best_model, 
                                                       max_len_of_signal_peptides=30,
                                                       training_column_name = 'MM_N_peptide_abundance')

In [48]:
best_predicted_SPs

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,predictions
756,M,V,L,V,W,W,S,L,W,A,...,-,-,-,-,-,-,-,-,-,0.187177
973,M,K,V,R,W,A,C,N,R,G,...,-,-,-,-,-,-,-,-,-,0.161975
355,M,V,R,P,S,A,L,G,G,A,...,-,-,-,-,-,-,-,-,-,0.145210
307,M,L,L,S,A,A,R,L,M,A,...,-,-,-,-,-,-,-,-,-,0.144589
575,M,L,L,I,A,A,I,S,F,L,...,-,-,-,-,-,-,-,-,-,0.142893
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,M,L,T,S,T,A,A,L,M,F,...,-,-,-,-,-,-,-,-,-,0.102710
984,M,K,V,T,V,A,I,L,M,L,...,-,-,-,-,-,-,-,-,-,0.102656
269,M,L,V,F,S,L,I,L,V,A,...,-,-,-,-,-,-,-,-,-,0.102220
565,M,M,R,F,S,W,S,T,L,G,...,-,-,-,-,-,-,-,-,-,0.102022


In [49]:
best_predicted_SPs.to_csv('../data/05_best_signal_peptides/100/best_signal_peptides_i_1000.csv', index=False)


In [50]:
best_predicted_SPs = pd.read_csv('../data/05_best_signal_peptides/100/best_signal_peptides_i_1000.csv', index_col=False)

In [51]:
best_predicted_SPs

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,predictions
0,M,V,L,V,W,W,S,L,W,A,...,-,-,-,-,-,-,-,-,-,0.187177
1,M,K,V,R,W,A,C,N,R,G,...,-,-,-,-,-,-,-,-,-,0.161975
2,M,V,R,P,S,A,L,G,G,A,...,-,-,-,-,-,-,-,-,-,0.145210
3,M,L,L,S,A,A,R,L,M,A,...,-,-,-,-,-,-,-,-,-,0.144589
4,M,L,L,I,A,A,I,S,F,L,...,-,-,-,-,-,-,-,-,-,0.142893
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,M,L,T,S,T,A,A,L,M,F,...,-,-,-,-,-,-,-,-,-,0.102710
96,M,K,V,T,V,A,I,L,M,L,...,-,-,-,-,-,-,-,-,-,0.102656
97,M,L,V,F,S,L,I,L,V,A,...,-,-,-,-,-,-,-,-,-,0.102220
98,M,M,R,F,S,W,S,T,L,G,...,-,-,-,-,-,-,-,-,-,0.102022


In [52]:
cols = [str(i) for i in range(0,30)]

In [53]:
best_predicted_SPs['sequence'] = best_predicted_SPs[cols].agg(''.join, axis=1)
best_predicted_SPs

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,predictions,sequence
0,M,V,L,V,W,W,S,L,W,A,...,-,-,-,-,-,-,-,-,0.187177,MVLVWWSLWALGAQVALPSVV---------
1,M,K,V,R,W,A,C,N,R,G,...,-,-,-,-,-,-,-,-,0.161975,MKVRWACNRGLLVGATLAAV----------
2,M,V,R,P,S,A,L,G,G,A,...,-,-,-,-,-,-,-,-,0.145210,MVRPSALGGASGLQSAAPPH----------
3,M,L,L,S,A,A,R,L,M,A,...,-,-,-,-,-,-,-,-,0.144589,MLLSAARLMALALTALAA------------
4,M,L,L,I,A,A,I,S,F,L,...,-,-,-,-,-,-,-,-,0.142893,MLLIAAISFLCGLIANAA------------
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,M,L,T,S,T,A,A,L,M,F,...,-,-,-,-,-,-,-,-,0.102710,MLTSTAALMFSALVATLP------------
96,M,K,V,T,V,A,I,L,M,L,...,-,-,-,-,-,-,-,-,0.102656,MKVTVAILMLLALVAALAALM---------
97,M,L,V,F,S,L,I,L,V,A,...,-,-,-,-,-,-,-,-,0.102220,MLVFSLILVALGLKATAF------------
98,M,M,R,F,S,W,S,T,L,G,...,-,-,-,-,-,-,-,-,0.102022,MMRFSWSTLGPILQMSAM------------


In [54]:
best_predicted_SPs = best_predicted_SPs.replace(to_replace = '-', value ='')
best_predicted_SPs['sequence'] =  [string.replace("-", "") for string in list(best_predicted_SPs['sequence'])]

In [55]:
lenght_of_signal_peptides = []
for k,v in best_predicted_SPs.iterrows(): 
    #print(v['sequence'])
    lenght_of_signal_peptides.append(len(v['sequence']))
best_predicted_SPs['length'] = lenght_of_signal_peptides

In [56]:
best_predicted_SPs

,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,predictions,sequence,length
0,M,V,L,V,W,W,S,L,W,A,...,,,,,,,,0.187177,MVLVWWSLWALGAQVALPSVV,21
1,M,K,V,R,W,A,C,N,R,G,...,,,,,,,,0.161975,MKVRWACNRGLLVGATLAAV,20
2,M,V,R,P,S,A,L,G,G,A,...,,,,,,,,0.145210,MVRPSALGGASGLQSAAPPH,20
3,M,L,L,S,A,A,R,L,M,A,...,,,,,,,,0.144589,MLLSAARLMALALTALAA,18
4,M,L,L,I,A,A,I,S,F,L,...,,,,,,,,0.142893,MLLIAAISFLCGLIANAA,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,M,L,T,S,T,A,A,L,M,F,...,,,,,,,,0.102710,MLTSTAALMFSALVATLP,18
96,M,K,V,T,V,A,I,L,M,L,...,,,,,,,,0.102656,MKVTVAILMLLALVAALAALM,21
97,M,L,V,F,S,L,I,L,V,A,...,,,,,,,,0.102220,MLVFSLILVALGLKATAF,18
98,M,M,R,F,S,W,S,T,L,G,...,,,,,,,,0.102022,MMRFSWSTLGPILQMSAM,18


In [57]:
SPs_22 = lets_predict_signal_peptides[(lets_predict_signal_peptides[['length']]<23).all(axis=1)]
SPs_22

,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,predictions,sequence,length
0,M,V,L,V,W,W,S,L,W,A,...,,,,,,,,0.187177,MVLVWWSLWALGAQVALPSVV,21
1,M,K,V,R,W,A,C,N,R,G,...,,,,,,,,0.161975,MKVRWACNRGLLVGATLAAV,20
2,M,V,R,P,S,A,L,G,G,A,...,,,,,,,,0.145210,MVRPSALGGASGLQSAAPPH,20
3,M,L,L,S,A,A,R,L,M,A,...,,,,,,,,0.144589,MLLSAARLMALALTALAA,18
4,M,L,L,I,A,A,I,S,F,L,...,,,,,,,,0.142893,MLLIAAISFLCGLIANAA,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,M,L,T,S,T,A,A,L,M,F,...,,,,,,,,0.102710,MLTSTAALMFSALVATLP,18
96,M,K,V,T,V,A,I,L,M,L,...,,,,,,,,0.102656,MKVTVAILMLLALVAALAALM,21
97,M,L,V,F,S,L,I,L,V,A,...,,,,,,,,0.102220,MLVFSLILVALGLKATAF,18
98,M,M,R,F,S,W,S,T,L,G,...,,,,,,,,0.102022,MMRFSWSTLGPILQMSAM,18


In [58]:
SPs_22.to_csv('../data/05_best_signal_peptides/100/best_signal_peptides_i_1000_full_w_less_than22.csv', index=False)


In [59]:
h2o.shutdown()

H2O session _sid_9584 closed.


/var/folders/2f/lw3sfzbs7l7f_q1knzbtbwrr0000gp/T/ipykernel_845/2900054706.py:1: H2ODeprecationWarning: Deprecated, use ``h2o.cluster().shutdown()``.
  h2o.shutdown()
